Tool Calling:

Tool calling is the process where the llm(large language model) decides, during a conversation or tasks, that it  needs to use a specific tool (function) - and generate a structured output with:

- the name of the tool.
- and the argument to call it with.

The llm does not actually run the tool - it just suggests the tool and the input arguments. The actual execution is handled by Langchain or you.

In [ ]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [5]:
@tool
def multiply(a: int, b: int) -> int:
    """Given 2 numbers a and b this tool returns their product"""
    return a * b

In [6]:
print(multiply.invoke({"a": 3, "b": 3}))

print(multiply.name)
print(multiply.description)

9
multiply
Given 2 numbers a and b this tool returns their product


In [7]:
model = ChatGroq(model="llama-3.1-8b-instant", temperature=0.3)

In [9]:
model_with_tools = model.bind_tools([multiply])

In [10]:
model_with_tools.invoke("hi how are you?")

AIMessage(content="I'm functioning properly, thanks for asking. What can I help you with today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 235, 'total_tokens': 253, 'completion_time': 0.027645032, 'prompt_time': 0.01280051, 'queue_time': 0.05102015, 'total_time': 0.040445542}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_33e8adf159', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--e885332d-4d6c-4759-9566-65554be52286-0', usage_metadata={'input_tokens': 235, 'output_tokens': 18, 'total_tokens': 253})

In [ ]:
model_with_tools.invoke("cna you multiply 3 with 10")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'hpkqg1w6v', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 239, 'total_tokens': 258, 'completion_time': 0.030148918, 'prompt_time': 0.013189112, 'queue_time': 0.049199417, 'total_time': 0.04333803}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_e32974efee', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--31ccb4f9-2841-4033-8114-0c01541553a9-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'hpkqg1w6v', 'type': 'tool_call'}], usage_metadata={'input_tokens': 239, 'output_tokens': 19, 'total_tokens': 258})

In [14]:
model_with_tools.invoke("cna you multiply 3 and 5 and 10").tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 5},
 'id': 'g525xtctd',
 'type': 'tool_call'}

Tool Execution:

Tool execution is the step where the actual python function (tool) is run using the input arguments that the llm suggested during tool calling.

In [15]:
result = model_with_tools.invoke("can you multiply 3 and 10")

In [19]:
result.tool_calls[0]

{'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': '75zfzx6yq',
 'type': 'tool_call'}

In [18]:
result.tool_calls[0]['args']

{'a': 3, 'b': 10}

In [20]:
multiply.invoke(result.tool_calls[0]['args'])

30

In [21]:
multiply.invoke({'name': 'multiply',
 'args': {'a': 3, 'b': 10},
 'id': '75zfzx6yq',
 'type': 'tool_call'})

ToolMessage(content='30', name='multiply', tool_call_id='75zfzx6yq')

In [22]:
multiply.invoke(result.tool_calls[0])

ToolMessage(content='30', name='multiply', tool_call_id='75zfzx6yq')

Below is the nice code:

In [ ]:
query = HumanMessage("can you multiply 3 with 1000")

In [34]:
messages = [query]

In [33]:
result = model_with_tools.invoke(messages)

In [26]:
messages.append(result)

In [32]:
messages

[HumanMessage(content='can you multiply 3 and 10', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'bmtwb3sqz', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 238, 'total_tokens': 257, 'completion_time': 0.028879651, 'prompt_time': 0.012923715, 'queue_time': 0.049791115, 'total_time': 0.041803366}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_7b3cfae3af', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7efd1803-8b15-488e-9ce1-1e9a7c160a4a-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'bmtwb3sqz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 238, 'output_tokens': 19, 'total_tokens': 257}),
 ToolMessage(content='30', name='multiply', tool_call_id='bmtwb3sqz')]

In [35]:
tool_result = multiply.invoke(result.tool_calls[0])

In [36]:
messages.append(tool_result)

In [37]:
messages

[HumanMessage(content='can you multiply 3 and 10', additional_kwargs={}, response_metadata={}),
 ToolMessage(content='30', name='multiply', tool_call_id='6jk57nx7t')]

In [38]:
model_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'mh9bshdd7', 'function': {'arguments': '{"a":3,"b":10}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 245, 'total_tokens': 264, 'completion_time': 0.023521107, 'prompt_time': 0.013432691, 'queue_time': 0.047684479, 'total_time': 0.036953798}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--63569e5c-326b-4408-95e8-c2684484993e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 10}, 'id': 'mh9bshdd7', 'type': 'tool_call'}], usage_metadata={'input_tokens': 245, 'output_tokens': 19, 'total_tokens': 264})